In [ ]:
%matplotlib inline
import d2lzh as d2l
from mxnet import autograd, nd
#获取数据集
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition  # 这里应用了广播机制

def net(X):
    return softmax(nd.dot(X.reshape((-1, num_inputs)), W) + b)

def cross_entropy(y_hat, y):
    return -nd.pick(y_hat, y).log()

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        y = y.astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum().asscalar()
        n += y.size
    return acc_sum / n
#定义超参数
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, trainer=None):
    #定义模型参数，附上梯度
    for epoch in range(num_epochs):
        # 初始化损失
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        #对每个样本进行训练
        for X ,y in train_iter:
            #作用域内的计算才会被用于反向传播
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
            l.backward()#再缩进和现在没区别
            #参数优化器配置，更新模型参数
            if trainer is None:
                d2l.sgd(params, lr, batch_size)  
            y = y.astype('float32')
            train_l_sum += l.sum().asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
            % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))     
        
train_ch3(net,train_iter,test_iter,cross_entropy,num_epochs, batch_size,[W,b],lr)